In [1]:
import awswrangler as wr
import boto3
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
# variáveis de entrada
nome_bucket = 'elasticbeanstalk-us-east-2-047710767346'
subpasta = 'Case_A3'
pasta_origem = f's3://{nome_bucket}/{subpasta}/arquivos_extraidos/'
pasta_destino = f's3://{nome_bucket}/{subpasta}/arquivos_extraidos_parquet/'

s3 = boto3.client('s3')

In [3]:
# Lista todos os arquivos na pasta de origem
files = wr.s3.list_objects(pasta_origem)


In [4]:
files

['s3://elasticbeanstalk-us-east-2-047710767346/Case_A3/arquivos_extraidos/1995_microdados_censo_da_educacao_superior_1995/Microdados do Censo da Educaç╞o Superior 1995/DADOS/GRADUACAO_PRESENCIAL.CSV',
 's3://elasticbeanstalk-us-east-2-047710767346/Case_A3/arquivos_extraidos/1995_microdados_censo_da_educacao_superior_1995/Microdados do Censo da Educaç╞o Superior 1995/DADOS/INSTITUICAO.CSV',
 's3://elasticbeanstalk-us-east-2-047710767346/Case_A3/arquivos_extraidos/1995_microdados_censo_da_educacao_superior_1995/Microdados do Censo da Educaç╞o Superior 1995/DADOS/MD5_microdados_ed_superior_1995.TXT',
 's3://elasticbeanstalk-us-east-2-047710767346/Case_A3/arquivos_extraidos/1995_microdados_censo_da_educacao_superior_1995/Microdados do Censo da Educaç╞o Superior 1995/Leia-me/Leia_me Superior_1995.pdf',
 's3://elasticbeanstalk-us-east-2-047710767346/Case_A3/arquivos_extraidos/1996_microdados_censo_da_educacao_superior_1996/Microdados do Censo da Educaç╞o Superior 1996/DADOS/GRADUACAO_PRESENC

In [5]:
# loop  filtrar listar arquivos CSV target (pasta dados)
for file in files:
    # filtra apenas arquivos CSV
    if file.lower().endswith('.csv'):
        # divide o caminho para obter os elementos do diretório
        components = file.split('/')
        
        # procura por uma pasta chamada "dados", "Dados" ou "DADOS"
        dados_folder_index = next((i for i, component in enumerate(components) if component.lower() == 'dados'), None)

        # se encontrar a pasta "dados", processa o arquivo
        if dados_folder_index is not None and dados_folder_index < len(components) - 1:
            # obtém o caminho para o arquivo dentro da pasta "dados"
            path_inside_dados = '/'.join(components[dados_folder_index + 1:])

            # imprime o nome dos arquivos .csv dentro da pasta específica "dados"
            print(path_inside_dados)


GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
FORME_DISTANCIA.CSV
FORME_PRESENCIAL.CSV
GRADUACAO_DISTANCIA.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
SECOMPLE_DISTANCIA.CSV
SECOMPLE_PRESENCIAL.CSV
FORME_DISTANCIA.CSV
FORME_PRESENCIAL.CSV
GRADUACAO_DISTANCIA.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
SECOMPLE_PRESENCIAL.CSV
FORME_DISTANCIA.CSV
FORME_PRESENCIAL.CSV
GRADUACAO_DISTANCIA.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
SECOMPLE_PRESENCIAL.CSV
FORME_DISTANCIA.CSV
FORME_PRESENCIAL.CSV
GRADUACAO_DISTANCIA.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
SECOMPLE_PRESENCIAL.CSV
FORME_DISTANCIA.CSV
FORME_PRESENCIAL.CSV
GRADUACAO_DISTANCIA.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
SECOMPLE_PRESENCIAL.CSV
FORME_DISTANCIA.CSV
FORME_PRESENCIAL.CSV
GRADUACAO_DISTANCIA.CSV
GRADUACAO_PRESENCIAL.CSV
INSTITUICAO.CSV
SECOMPLE_PRESENC

In [6]:
## teste para 1 arquivo
file = f's3://{nome_bucket}/{subpasta}/arquivos_extraidos/1995_microdados_censo_da_educacao_superior_1995/Microdados do Censo da Educaç╞o Superior 1995/DADOS/GRADUACAO_PRESENCIAL.CSV'


# lê o arquivo e armazena em um df
df = wr.s3.read_csv(file, sep = '|', encoding = 'latin-1')

# converte o nome das colunas para minúsculo
df.columns = df.columns.str.lower()
            
df.head()

,mascara,nr_ano,cd_curso_seec,cd_curso,codmunic,eh_capital,no_regiao,sg_uf_curso,no_uf,no_curso,...,qt_ingr_outros_femi_diurno,qt_ingr_outros_masc_diurno,qt_vagas_noturno,qt_insc_1opc_noturno,qt_insc_2opc_noturno,qt_aprovados_noturno,qt_ingr_vest_femi_noturno,qt_ingr_vest_masc_noturno,qt_ingr_outros_femi_noturno,qt_ingr_outros_masc_noturno
0,1,1995,301,2020000,NaN,N,CENTRO-OESTE,GO,Goias,CIENCIA DA COMPUTACAO,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,302,2066000,NaN,N,SUDESTE,MG,Minas Gerais,LETRAS,...,0,0,60,58,0,58,42,4,1,0
2,2,1995,303,2050000,NaN,N,SUDESTE,MG,Minas Gerais,ESTUDOS SOCIAIS,...,0,0,60,41,0,41,25,16,0,0
3,2,1995,301,2018000,NaN,N,SUDESTE,MG,Minas Gerais,CIENCIAS,...,0,0,60,50,0,50,36,4,1,0
4,3,1995,302,2021000,NaN,N,SUDESTE,RJ,Rio de Janeiro,CIENCIAS CONTABEIS,...,0,0,50,160,139,50,29,21,0,0


In [13]:
pasta_origem = f's3://{nome_bucket}/{subpasta}/arquivos_extraidos'
pasta_destino = f's3://{nome_bucket}/{subpasta}/arquivos_extraidos_parquet'


In [12]:
# salva o arquivo como parquet na pasta de destino

caminho_e_nome_arquivo = f'{pasta_destino}/teste2.parquet'
wr.s3.to_parquet(df, path=caminho_e_nome_arquivo, dataset=True)
print(f"Arquivo convertido com sucesso: {caminho_e_nome_arquivo}")

Arquivo convertido com sucesso: s3://elasticbeanstalk-us-east-2-047710767346/Case_A3/arquivos_extraidos_parquet/teste2.parquet


In [18]:
# Storing data on Data Lake
wr.s3.to_parquet(
    df=df,
    path= f'{pasta_destino}/',
    dataset=True,
)

# Retrieving the data directly from Amazon S3
df = wr.s3.read_parquet(f'{pasta_destino}/1a125d1c630848cdb022a84a1f881114.snappy.parquet', dataset=True)

/home/lula/.local/lib/python3.9/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by mode='default'.
  return cls.dispatch_func(func)(*args, **kw)


ArrowTypeError: Unable to merge: Field c3211 has incompatible types: double vs int64

In [19]:
import pyarrow as pa

# Suponha que df seja o DataFrame lido do CSV
schema = pa.Schema.from_pandas(df)

# Salva o arquivo como Parquet na pasta de destino, especificando o esquema
wr.s3.to_parquet(
    df=df,
    path=caminho_e_nome_arquivo,
    dataset=True,
    schema=schema,  # Especifica o esquema ao escrever
)

print(f"Arquivo convertido com sucesso: {caminho_e_nome_arquivo}")


TypeError: got an unexpected keyword argument 'schema'

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

# Suponha que df seja o DataFrame lido do CSV
schema = pa.Schema.from_pandas(df)

# Salva o arquivo CSV com o esquema especificado
wr.s3.to_csv(df=df, path='s3://seu-bucket/seu-arquivo.csv', index=False, schema=schema)

# Lê o arquivo CSV com o esquema especificado
df = wr.s3.read_csv('s3://seu-bucket/seu-arquivo.csv', schema=schema)

# Salva o arquivo como Parquet
wr.s3.to_parquet(df=df, path=f's3://{pasta_destino}/1a125d1c630848cdb022a84a1f881114.snappy.parquet', dataset=True)


In [20]:
df.head()

,mascara,nr_ano,cd_curso_seec,cd_curso,codmunic,eh_capital,no_regiao,sg_uf_curso,no_uf,no_curso,...,qt_ingr_outros_femi_diurno,qt_ingr_outros_masc_diurno,qt_vagas_noturno,qt_insc_1opc_noturno,qt_insc_2opc_noturno,qt_aprovados_noturno,qt_ingr_vest_femi_noturno,qt_ingr_vest_masc_noturno,qt_ingr_outros_femi_noturno,qt_ingr_outros_masc_noturno
0,1,1995,301,2020000,NaN,N,CENTRO-OESTE,GO,Goias,CIENCIA DA COMPUTACAO,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,302,2066000,NaN,N,SUDESTE,MG,Minas Gerais,LETRAS,...,0,0,60,58,0,58,42,4,1,0
2,2,1995,303,2050000,NaN,N,SUDESTE,MG,Minas Gerais,ESTUDOS SOCIAIS,...,0,0,60,41,0,41,25,16,0,0
3,2,1995,301,2018000,NaN,N,SUDESTE,MG,Minas Gerais,CIENCIAS,...,0,0,60,50,0,50,36,4,1,0
4,3,1995,302,2021000,NaN,N,SUDESTE,RJ,Rio de Janeiro,CIENCIAS CONTABEIS,...,0,0,50,160,139,50,29,21,0,0


In [22]:
# Retrieving the data directly from Amazon S3
df = wr.s3.read_parquet(f's3://{nome_bucket}/{subpasta}/arquivos_extraidos_parquet/1a125d1c630848cdb022a84a1f881114.snappy.parquet', dataset=True)

Exception: Object s3://elasticbeanstalk-us-east-2-047710767346/Case_A3/arquivos_extraidos_parquet/1a125d1c630848cdb022a84a1f881114.snappy.parquet is not under the root path (s3://elasticbeanstalk-us-east-2-047710767346/Case_A3/arquivos_extraidos_parquet/1a125d1c630848cdb022a84a1f881114.snappy.parquet/).